In [1]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 3.9 MB/s eta 0:00:00


## Versão do video


In [2]:
import os
os.environ['GROQ_API_KEY'] = "GROQ_API_KEY"


In [ ]:

from openai import OpenAI
client = OpenAI(
    api_key=os.environ.get("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)

response = client.responses.create(
    input="Explain the importance of fast language models",
    model="openai/gpt-oss-20b",
)
print(response.output_text)


In [4]:
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = self.client.chat.completions.create(
            messages=self.messages,
            model="openai/gpt-oss-20b",
        )
        return completion.choices[0].message.content


In [ ]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:

e.g. calculate: 4 * 7 / 3

Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:

e.g. wikipedia: Django

Returns a summary from searching Wikipedia

simon_blog_search:

e.g. simon_blog_search: Django

Search Simon's blog for that term
Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France

PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris
""".strip()

In [ ]:
def calculate(operation):
    return eval(operation)

def get_plamet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "mars":
            return 6.39e23
        case "jupiter":
            return 1.898e27
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "neptune":
            return 1.024e26
        case "mercury":
            return 3.301e23
        case "venus":
            return 4.867e24
        case "sun":
            return 1.989e30

In [ ]:
neil_tyson = Agent(client, system_prompt)

In [ ]:
result = neil_tyson("What is the mass of the sun times 2?")
print(result)

BadRequestError: Error code: 400 - {'error': {'message': 'Tool choice is none, but model called a tool', 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '{"name": "python", "arguments": # no code yet}'}}

In [ ]:
import re

def loop(max_iterations=10, query: str = ""):
    agent = Agent(client=client, system=system_prompt)
    tools = ["calculate", "get_planet_mass"]
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break

loop(query="What is the mass of Earth plus the mass of Saturn and all of that times 2?")

## Corrigindo a versão do vídeo

Devido a evolução dos modelo e das bibliotecas usadas, parece que houve um descompasso entre as instruções dadas (*system_prompt*) ao modelo e a configuração do código apresentadas no vídeo.

1. inicialmente o erro parece indicar uma falha de *parsing*, que acontece devido ao uso do prompt de ReAct sem ter um script que fizesse a formatação correta das requisições;
2. posteriormente, o erro *BadRequest* surgiu quando o modelo tentou utilizar o recurso nativo de chamada de funções, mas foi bloqueado pela API porque as ferramentas (*tools*) não haviam sido declaradas formalmente na requisição, criando uma incompatibilidade entre a intenção do modelo e os parâmetros fornecidos.

In [20]:
import json
import re
from openai import OpenAI

client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key="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"
)

In [21]:
def calculate(expression):
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Erro: {e}"

# função add para teste
def wikipedia_search(query):
    return f"Resultado simulado da Wikipedia para: {query}"

def get_planet_mass(planet):
    masses = {
        "earth": 5.972e24,
        "mars": 6.39e23,
        "jupiter": 1.898e27,
        "saturn": 5.683e26,
        "uranus": 8.681e25,
        "neptune": 1.024e26,
        "mercury": 3.301e23,
        "venus": 4.867e24,
        "sun": 1.989e30
    }
    return str(masses.get(planet.lower(), "Massa desconhecida"))


In [14]:
# log criando manualmente pois não consegui acessar modelo.messages
def save_log_to_json(messages, filename="historico_conversa.json"):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(messages, f, indent=4, ensure_ascii=False, default=str)
    print(f"Log salvo em {filename}")

#### lista de ferramentas disponíveis

In [22]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Calculates a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The math expression to evaluate, e.g., '2 + 2'"
                    }
                },
                "required": ["expression"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "wikipedia_search",
            "description": "Search for a term on Wikipedia",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_planet_mass",
            "description": "Get the mass of a planet in kg",
            "parameters": {
                "type": "object",
                "properties": {
                    "planet": {"type": "string", "description": "The name of the planet, e.g., 'Mars'"}
                },
                "required": ["planet"],
            },
        },
    }
]

available_functions = {
    "calculate": calculate,
    "wikipedia_search": wikipedia_search,
    "get_planet_mass": get_planet_mass,
}

#### definindo novo modelo

In [28]:
# novo modelo, atualizado com base nos modelos atuais disponíveis no site do groq
def steve_magal(question):
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Use the available tools to answer questions correctly."},
        {"role": "user", "content": question}
    ]

    # criando log manual para checagem posterior
    save_log_to_json(messages)

    print(f"--- Pergunta: {question} ---")

    # criando instancia do cliente
    response = client.chat.completions.create(
        model="openai/gpt-oss-20b", # modelo encontrado no site oficial do GROQ
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        print("--> O modelo decidiu usar ferramentas!")

        messages.append(response_message)

        # add informação no log
        save_log_to_json(messages)

        # iterando sobre as chamadas
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)

            print(f"--> Executando: {function_name} com args {function_args}")

            if function_name == "calculate":
                function_response = function_to_call(expression=function_args.get("expression"))
            elif function_name == "wikipedia_search":
                function_response = function_to_call(query=function_args.get("query"))
            elif function_name == "get_planet_mass":
                function_response = function_to_call(planet=function_args.get("planet"))

            print(f"--> Resultado: {function_response}")

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

           # add informação no log
            save_log_to_json(messages)

        # segunda chamada do modelo, para verificação das respostas
        second_response = client.chat.completions.create(
            model="openai/gpt-oss-20b",
            messages=messages,
        )
        # add informação no log
        save_log_to_json(messages)
        return second_response.choices[0].message.content

    else:
        return response_message.content

#### prompt corrigido

In [10]:
# prompt simplificado para evitar o erro de parser
system_prompt = """
You are a helpful and knowledgeable AI assistant.
Answer the user's questions correctly and concisely.
If the question is complex, explain your reasoning step-by-step before providing the final answer.
""".strip()

#### chamada do modelo

In [15]:
res = steve_magal("What is the mass of earth times 5?")
print(f"\nRESPOSTA FINAL:\n{res}")

Log salvo em historico_conversa.json
--- Pergunta: What is the mass of earth times 5? ---
--> O modelo decidiu usar ferramentas!
Log salvo em historico_conversa.json
--> Executando: calculate com args {'expression': '5.972e24 * 5'}
--> Resultado: 2.9860000000000004e+25
Log salvo em historico_conversa.json
Log salvo em historico_conversa.json

RESPOSTA FINAL:
The mass of the Earth is approximately \(5.972 \times 10^{24}\) kg.  
Multiplying that by 5 gives:

\[
5.972 \times 10^{24}\ \text{kg} \times 5 \approx 2.986 \times 10^{25}\ \text{kg}.
\]

So, the result is roughly **\(2.986 \times 10^{25}\) kg**.


#### lendo o histórico de "pensamento" do modelo

In [16]:
def ler_log(nome_arquivo="historico_conversa.json"):
    try:
        with open(nome_arquivo, "r", encoding="utf-8") as f:
            dados = json.load(f)
            print(json.dumps(dados, indent=4, ensure_ascii=False))
    except FileNotFoundError:
        print("Arquivo de log não encontrado. Rode o agente primeiro.")

ler_log()

[
    {
        "role": "system",
        "content": "You are a helpful assistant. Use the available tools to answer questions correctly."
    },
    {
        "role": "user",
        "content": "What is the mass of earth times 5?"
    },
    "ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='fc_fe6cf9fe-8554-44d1-abd8-9bd5b42beab9', function=Function(arguments='{\"expression\":\"5.972e24 * 5\"}', name='calculate'), type='function')], reasoning='The user: \"What is the mass of earth times 5?\" They want mass of Earth multiplied by 5. We need to compute that. Mass of Earth: about 5.972 × 10^24 kg. Times 5: 2.986 × 10^25 kg. We could give that. Maybe use calculate tool? But it\\'s a simple multiplication. Could use the function? The calculate function expects an expression. Let\\'s use it.')",
    {
        "tool_call_id": "fc_fe6cf9fe-8554-44d1-abd8-9bd5b42beab9",
  

### executando o agente em loop

In [34]:
# função steve_magal alterada para retirar a "memória inicial"
# e permitir a passagem de informação através das interações em loop

def novo_steve_magal(messages):

    save_log_to_json(messages)

    response = client.chat.completions.create(
        model="openai/gpt-oss-20b",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        print("   [Pensamento]: O modelo decidiu usar ferramentas!")
        messages.append(response_message)

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)

            print(f"   [Ação]: Executando {function_name} com args {function_args}")

            if function_name == "calculate":
                function_response = function_to_call(expression=function_args.get("expression"))
            elif function_name == "wikipedia_search":
                function_response = function_to_call(query=function_args.get("query"))
            elif function_name == "get_planet_mass":
                function_response = function_to_call(planet=function_args.get("planet"))

            print(f"   [Observação]: Resultado: {function_response}")

            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": str(function_response),
            })
            save_log_to_json(messages)

        return "CONTINUE"

    else:
        return response_message.content

In [35]:
def loop_de_raciocinio(question, max_steps=10):
    print(f"--- Iniciando tarefa complexa: {question} ---")

    # "memória"
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Use tools step-by-step to answer."},
        {"role": "user", "content": question}
    ]

    step = 0
    while step < max_steps:
        step += 1
        print(f"\n>> Passo de Raciocínio {step}")

        # agora a função recebe a "memória" da interação passada
        resultado = novo_steve_magal(messages)

        if resultado == "CONTINUE":
            continue
        else:
            return resultado

    return "Limite de passos atingido sem resposta final."

pergunta = "What is the mass of Earth plus the mass of Mars, and multiply the result by 2?"
resposta_final = loop_de_raciocinio(pergunta)

print(f"\nRESPOSTA FINAL DO AGENTE:\n{resposta_final}")

--- Iniciando tarefa complexa: What is the mass of Earth plus the mass of Mars, and multiply the result by 2? ---

>> Passo de Raciocínio 1
Log salvo em historico_conversa.json
   [Pensamento]: O modelo decidiu usar ferramentas!
   [Ação]: Executando get_planet_mass com args {'planet': 'Earth'}
   [Observação]: Resultado: 5.972e+24
Log salvo em historico_conversa.json

>> Passo de Raciocínio 2
Log salvo em historico_conversa.json
   [Pensamento]: O modelo decidiu usar ferramentas!
   [Ação]: Executando get_planet_mass com args {'planet': 'Mars'}
   [Observação]: Resultado: 6.39e+23
Log salvo em historico_conversa.json

>> Passo de Raciocínio 3
Log salvo em historico_conversa.json

RESPOSTA FINAL DO AGENTE:
The combined mass of Earth and Mars is approximately \(6.611 \times 10^{24}\,\text{kg}\).  
Multiplying that result by 2 gives about:

\[
1.322 \times 10^{25}\;\text{kg}
\]


In [36]:
resposta_final = loop_de_raciocinio(input("Faça uma pergunta ao Steve Magal: "))


Faça uma pergunta ao Steve Magal: Consegue me responder em português?
--- Iniciando tarefa complexa: Consegue me responder em português? ---

>> Passo de Raciocínio 1
Log salvo em historico_conversa.json
